In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
!pip install pyarrow

     |████████████████████████████████| 42.1 MB 76 kB/s s eta 0:00:01     |███████████████████████████████▍| 41.3 MB 12.6 MB/s eta 0:00:01


### Q1. Downloading the data

In [ ]:
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [15]:
len(df_jan.columns)

19

Re: The data from January has 19 columns

### Q2. Computing duration

In [25]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]

In [28]:
df_jan["duration"].std().seconds / 60

42.583333333333336

Re: The standard deviation of the trips duration in January is 42.59 minutes

### Q3. Dropping outliers